## Step-1: Importing Required Libraries

In [1]:
# Multilingual Support
from langdetect import detect
from googletrans import Translator
# Transformers (TinyLlama), speech recognition, TTS
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import speech_recognition as sr
from gtts import gTTS
import IPython.display as ipd
import tempfile
import torch
import nltk
import random
import pygame  # This works well to auto-play audio instantly
import time
import tempfile
from gtts import gTTS


# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

/Users/yaswanthganapathi/anaconda3/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


pygame 2.5.1 (SDL 2.28.2, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yaswanthganapathi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yaswanthganapathi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Step-2: Loading TinyLlama Model

In [2]:
# Load TinyLlama model from Hugging Face
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

# Setup pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Some parameters are on the meta device because they were offloaded to the disk.


## Step-3: Loading the Patient datasets

In [3]:
# Load datasets using file paths from the uploaded files
df_profiles = pd.read_csv("Created_Patients_Data/allergies_and_diet.csv")
df_encounters = pd.read_csv("Created_Patients_Data/clinical_encounters.csv")
df_labs = pd.read_csv("Created_Patients_Data/lab_results_current.csv")
df_mental = pd.read_csv("Created_Patients_Data/mental_health_mood_current.csv")
df_patients = pd.read_csv("Created_Patients_Data/patient_profiles.csv")
df_pghealth = pd.read_csv("Created_Patients_Data/pg_health_data_current.csv")

# Confirm loading by showing column names
print("Column names from each dataset:")
print("df_profiles:", df_profiles.columns.tolist())
print("df_encounters:", df_encounters.columns.tolist())
print("df_labs:", df_labs.columns.tolist())
print("df_mental:", df_mental.columns.tolist())
print("df_patients:", df_patients.columns.tolist())
print("df_pghealth:", df_pghealth.columns.tolist())


Column names from each dataset:
df_profiles: ['patient_id', 'food_allergies', 'drug_allergies', 'diet_type', 'avoid_foods', 'food_intolerances', 'patient_name']
df_encounters: ['patient_id', 'admission_date', 'discharge_date', 'diagnoses', 'medications', 'discharge_instructions', 'care_plan_goals', 'patient_name']
df_labs: ['patient_id', 'lab_date', 'BNP', 'glucose', 'hemoglobin', 'creatinine', 'patient_name', 'interpretation']
df_mental: ['patient_id', 'day', 'mood', 'stress_level', 'depression_score', 'patient_name']
df_patients: ['patient_id', 'name', 'age', 'gender', 'date_of_birth', 'preferred_language', 'reading_level', 'address', 'patient_name']
df_pghealth: ['patient_id', 'day', 'weight', 'systolic_bp', 'diastolic_bp', 'heart_rate', 'steps', 'sleep_duration', 'sleep_quality', 'patient_name']


## Step-4: Voice Input and Output 

In [4]:
# Capture voice from microphone
def listen_to_user():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎤 Listening...")
        audio = recognizer.listen(source)
        try:
            user_input = recognizer.recognize_google(audio)
            print(f"🗣️ You said: {user_input}")
            return user_input
        except:
            print("❌ Sorry, I couldn't understand that.")
            return ""

# Use gTTS to speak response
def speak_response(text):
    tts = gTTS(text=text, lang='en')
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
        tts.save(fp.name)
        pygame.mixer.init()
        pygame.mixer.music.load(fp.name)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            time.sleep(0.1)

## Step-5: Selecting Patient as per the Dataset

In [5]:
def select_patient():
    print("🩺 Available Patients:")
    for idx, row in df_patients.iterrows():
        print(f"{row['patient_id']}: {row['name']} ({row['preferred_language']})")
    
    while True:
        patient_id = input("\nEnter the Patient ID to continue (e.g., P001): ").strip()
        if patient_id in df_patients['patient_id'].values:
            return patient_id
        else:
            print("❌ Invalid Patient ID. Please try again.")


## Step-6: Building Patient Context

In [7]:
def build_patient_context(patient_id):
    profile = df_patients[df_patients['patient_id'] == patient_id].iloc[0]
    allergies = df_profiles[df_profiles['patient_id'] == patient_id]
    mental = df_mental[df_mental['patient_id'] == patient_id]
    labs = df_labs[df_labs['patient_id'] == patient_id]
    encounters = df_encounters[df_encounters['patient_id'] == patient_id]
    notes = df_pghealth[df_pghealth['patient_id'] == patient_id]

    context = f"""
🧑 Patient Info:
Name: {profile['name']}
Age: {profile['age']}, Gender: {profile['gender']}
Preferred Language: {profile['preferred_language']}
Reading Level: {profile['reading_level']}
Address: {profile['address']}

🌿 Allergies & Diet:
Food Allergies: {', '.join(allergies['food_allergies'].dropna().unique())}
Drug Allergies: {', '.join(allergies['drug_allergies'].dropna().unique())}
Diet Type: {', '.join(allergies['diet_type'].dropna().unique())}
Avoid Foods: {', '.join(allergies['avoid_foods'].dropna().unique())}
Food Intolerances: {', '.join(allergies['food_intolerances'].dropna().unique())}

🧠 Mental Health:
Recent Mood(s): {', '.join(mental['mood'].dropna().unique())}
Stress Level(s): {', '.join(map(str, mental['stress_level'].dropna().unique()))}
Depression Score(s): {', '.join(map(str, mental['depression_score'].dropna().unique()))}

🧪 Lab Results:
{"; ".join(labs['interpretation'].dropna().unique())}

📋 Clinical Encounters:
"""
    for _, row in encounters.iterrows():
        context += f"- Diagnosed with {row['diagnoses']} | Medications: {row['medications']}\n"

    context += "\n📊 Daily Health Notes:\n"
    for _, row in notes.iterrows():
        context += f"- Date: {row['day']}, Sleep: {row['sleep_quality']} hrs, Steps: {row['steps']}, Heart Rate: {row['heart_rate']} bpm\n"

    return context


In [6]:
def build_patient_context(patient_id):
    # Get patient profile
    profile = df_patients[df_patients['patient_id'] == patient_id].iloc[0]
    allergies_info = df_profiles[df_profiles['patient_id'] == patient_id].iloc[0]
    moods = df_mental[df_mental['patient_id'] == patient_id]['mood'].dropna().unique()
    lab_interpretations = df_labs[df_labs['patient_id'] == patient_id]['interpretation'].dropna().unique()
    encounters = df_encounters[df_encounters['patient_id'] == patient_id]
    notes = df_pghealth[df_pghealth['patient_id'] == patient_id]

    # Format context string
    context = f"""
👤 Patient Name: {profile['name']}
Age: {profile['age']}, Gender: {profile['gender']}, Language: {profile['preferred_language']}
Address: {profile['address']}

🍽️ Diet Type: {allergies_info['diet_type']}
Food Allergies: {allergies_info['food_allergies']}
Drug Allergies: {allergies_info['drug_allergies']}
Avoid Foods: {allergies_info['avoid_foods']}
Food Intolerances: {allergies_info['food_intolerances']}

🧠 Mental Health Moods: {', '.join(moods)}

🧪 Lab Interpretations:
- {'; '.join(lab_interpretations)}

🏥 Clinical Encounters:
"""
    for _, row in encounters.iterrows():
        context += f"  • Diagnoses: {row['diagnoses']}, Medications: {row['medications']}\n"
        context += f"    Care Goals: {row['care_plan_goals']}, Discharge Notes: {row['discharge_instructions']}\n"

    context += "\n📊 Personal Health Tracking:\n"
    for _, row in notes.iterrows():
        context += f"  • Date: {row['day']}, Weight: {row['weight']}kg, BP: {row['systolic_bp']}/{row['diastolic_bp']} mmHg, Sleep: {row['sleep_duration']}h, Mood: {row['sleep_quality']}\n"

    return context.strip()


## Step-7: Loop to run the Clinical Assistant

In [10]:
def run_virtual_nurse():
    patient_id = select_patient()
    context = build_patient_context(patient_id)

    print("\n🩺 Virtual Nurse is ready. Say 'stop' to end the conversation.\n")
    
    # Greeting messages (randomized)
    greetings = [
        "Hi, I'm your Clinical Assistant. How can I help you today?",
        "Hello, I'm here to support you with your health. What's on your mind?",
        "Hi there! I'm your assistant nurse. Feel free to ask me anything health related.",
        "Welcome! I’m your virtual healthcare assistant. Let’s begin. Say 'stop' to exit."
    ]
    greeting = random.choice(greetings)
    
    print(f"🩺 Virtual Nurse: {greeting}")
    speak_response(greeting)

    while True:
        user_query = listen_to_user()

        if user_query.lower() in ["stop", "exit", "quit", "thank you exit", "thank you stop", "thank you quit", "please stop", "please exit", "please quit"]:
            goodbye = "Okay. Take care and stay healthy!"
            print("👋 Virtual Nurse:", goodbye)
            speak_response(goodbye)
            break

        if user_query:
            # Full prompt with personalized context
            prompt = f"""
You are a friendly and professional virtual nurse. Use the patient information provided to answer the user's health-related question in a supportive, conversational tone.

Patient Information:
{context}

Patient's Question: {user_query}

Virtual Nurse:"""

            # Generate response using TinyLlama
            response = generator(
                prompt,
                max_new_tokens=100,
                do_sample=True,
                top_p=0.9,
                temperature=0.7
            )[0]["generated_text"]

            # Clean the response
            final_response = response.replace(prompt, "").strip()

            print("🤖 Virtual Nurse:", final_response)
            speak_response(final_response)


In [11]:
# To Suppress warnings
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Step-8: Run the Assistant

In [13]:
# To start it
run_virtual_nurse()

🩺 Available Patients:
P001: Linda Thompson (English)
P002: Ravi Patel (Hindi)
P003: Sofia Ramirez (Spanish)

Enter the Patient ID to continue (e.g., P001): P002

🩺 Virtual Nurse is ready. Say 'stop' to end the conversation.

🩺 Virtual Nurse: Welcome! I’m your virtual healthcare assistant. Let’s begin. Say 'stop' to exit.
🎤 Listening...
🗣️ You said: hi I'm Ravi I'm having high temperature since 2 days what can I do
🤖 Virtual Nurse: Hi Ravi, Thank you for sharing your patient information. Based on the medical history you provided, it seems that you have high blood sugar and high BP. We recommend that you follow the following steps to manage your health:

1. Monitor your blood sugar and take your medications as directed by your healthcare provider. 2. Make sure to get enough sleep, as sleep is important for maintaining good health. 3. Avoid eating fried foods,
🎤 Listening...
🗣️ You said: okay what can I eat for dinner tonight can you suggest me as for my health records
🤖 Virtual Nurse: Su